In [3]:
import pandas as pd
from tqdm import tqdm

## Функции для анализа пересечения сегментов

In [4]:
def is_slf_intrsct(markup):    
    '''
    Функция is_slf_intrsct() определяет факт наличия пересечений
    сегментов внутри разметки markup. Требует один аргумент - разметку .deb.
    Возвращает True или False.
    '''
    
    # later should adapt for using lesser memory
    markup = markup.sort_values(by=[1], ascending=True)
    begin_list = list(markup[1])
    end_list = list(markup[2])
    slf_intrsct = False
    for zone_ in tqdm(range(1,len(begin_list))):
        if slf_intrsct == True:
            break
        if end_list[zone_ - 1] > begin_list[zone_]:
            slf_intrsct = True
    return slf_intrsct

def are_zones_intrsct(zone1, zone2):   
    '''
    Функция are_zones_intrsct() проверяет, пересекаются ли указанные 
    в агументах отрезки zone1 и zone2. Отрезки должны списками или кортежами длины 2.
    Возвращает True или False.
    '''
    
    begin = 0
    end = 1
    intrsct = False
    
    if zone1[begin] > zone2[begin]:
        zonet = zone2
        zone2 = zone1
        zone1 = zonet
    
    def less_cmpr(x1, x2):
        if (x1 - x2) <= 0:
            return True
        else:
            return False
    
    if not all([less_cmpr(zone1[begin], zone2[begin]),
                less_cmpr(zone1[end], zone2[end]),
                less_cmpr(zone1[end], zone2[begin])]):
        intrsct = True
    return intrsct

def get_intrscts(markup1, markup2, ask_if=False):    
    '''
    Функция get_intrscts() строит матричное представление графа, построенного на
    отношениях пересечений между отрезками разметок markup1 и markup2.
    Возвращает объект pd.DataFrame. Если аргумент ask_if=True, то возвращает только True или False.
    '''
    
    begin_list_1 = list(markup1[1])
    end_list_1 = list(markup1[2])
    begin_list_2 = list(markup2[1])
    end_list_2 = list(markup2[2])
    df_intrsct = pd.DataFrame(0, index=[i for i in range(1,len(begin_list_2)+1)],
                               columns=[i for i in range(1,len(begin_list_1)+1)])
    for zone1 in tqdm(range(len(begin_list_1))):
        for zone2 in range(len(begin_list_2)):
            if are_zones_intrsct([begin_list_1[zone1], end_list_1[zone1]],
                                 [begin_list_2[zone2], end_list_2[zone2]]):
                if ask_if:
                    return True
                df_intrsct.loc[zone2+1, zone1+1] = 1
    return df_intrsct

def get_comm_intrsct(segm1, segm2):    
    '''
    Функция get_comm_intrsct() возвращает отрезок, являющийся пересечением
    отрезков segm1 и segm2.
    '''
    
    begin = 0
    end = 1
    
    if not are_zones_intrsct(segm1, segm2):
        print('Segments do not intersect')
        return 0
    
    if segm2[begin] < segm1[begin]:
        segmt = segm2
        segm2 = segm1
        segm1 = segmt
    
    def less_cmpr(x1, x2):
        if (x1 - x2) <= 0:
            return True
        else:
            return False
    
    def which_case():
        nonlocal segm1, segm2
        if all([less_cmpr(segm2[begin], segm1[end]),
                less_cmpr(segm1[end], segm2[end])]):
            return 1
        if all([less_cmpr(segm2[begin], segm1[end]),
                less_cmpr(segm2[end], segm1[end])]):
            return 2
        else:
            return 0
    
    intrsct_case = which_case()
    if not intrsct_case:
        # print('Segments do not intersect correctly')
        return 0
    if intrsct_case == 1:
        return [segm2[begin], segm1[end]]
    if intrsct_case == 2:
        return [segm2[begin], segm2[end]]
    else:
        print('Smth is wrong')
        return 0
    
def build_intrscts(markup1, markup2):   
    '''
    Функция build_intrscts() строит .deb разметку на основе пересечения двух разметок
    markup1 и markup2. Возвращает объект pd.DataFrame.
    '''
    
    begin_list_1 = list(markup1[1])
    end_list_1 = list(markup1[2])
    begin_list_2 = list(markup2[1])
    end_list_2 = list(markup2[2])
    markup1_len = 0
    markup2_len = 0
    intrsct_len = 0
    
    intrsct_markup = pd.DataFrame(columns=[0, 1, 2])
    
    if markup2[0][0] != markup1[0][0]:
        if input('Different chromosomes if files. If no,\
        process will be stopped, else will be used name from 1-st file') == 'no':
            return 0
    chr_name = markup1[0][0]
    
    for zone1 in tqdm(range(len(begin_list_1))):
        for zone2 in range(len(begin_list_2)):
            segm1 = [begin_list_1[zone1], end_list_1[zone1]]
            segm2 = [begin_list_2[zone2], end_list_2[zone2]]
            markup1_len += segm1[1] - segm1[0]
            markup2_len += segm2[1] - segm2[0]
            if are_zones_intrsct(segm1, segm2):
                intrsct_zone = get_comm_intrsct(segm1, segm2)
                intrsct_markup = pd.concat([intrsct_markup,
                                            pd.DataFrame([[chr_name] + intrsct_zone],
                                                         columns=[0, 1, 2])], ignore_index=True)
                intrsct_len += intrsct_zone[1] - intrsct_zone[0]
    intrsct_markup.sort_values(by=[1], ascending=True, inplace=True)
    print('Общая разметка составляет ', 100*intrsct_len/markup1_len,'% и ',
          100*intrsct_len/markup2_len, '% от исходных')
    return intrsct_markup

In [3]:
print(is_slf_intrsct.__doc__)
print(are_zones_intrsct.__doc__)
print(get_intrscts.__doc__)
print(get_comm_intrsct.__doc__)
print(build_intrscts.__doc__)


    Функция is_slf_intrsct() определяет факт наличия пересечений
    сегментов внутри разметки markup. Требует один аргумент - разметку .deb.
    Возвращает True или False.
    

    Функция are_zones_intrsct() проверяет, пересекаются ли указанные 
    в агументах отрезки zone1 и zone2. Отрезки должны списками или кортежами длины 2.
    Возвращает True или False.
    

    Функция get_intrscts() строит матричное представление графа, построенного на
    отношениях пересечений между отрезками разметок markup1 и markup2.
    Возвращает объект pd.DataFrame. Если аргумент ask_if=True, то возвращает только True или False.
    

    Функция get_comm_intrsct() возвращает отрезок, являющийся пересечением
    отрезков segm1 и segm2.
    

    Функция build_intrscts() строит .deb разметку на основе пересечения двух разметок
    markup1 и markup2. Возвращает объект pd.DataFrame.
    


## Работа с реальными разметками

In [4]:
markup_tair = pd.read_csv("списки генов/ups2000_tair10.bed", sep="\t", header=None)

In [5]:
# markup_real_1 = pd.read_csv("DHS/DHS_Ath_flower_14_days_chr1.bed", sep="\t", header=None)
markup_real_2 = pd.read_csv("GSM2704255_ATAC_seq/GSM2704255_ATAC_chr1.bed", sep="\t", header=None)

### Читаем разметку tair. Проверяем избыточность, т.е. что есть самопересечения.

In [6]:
for chr_name in ['chr1', 'chr2', 'chr3', 'chr4', 'chr5']:
    print(is_slf_intrsct(markup_tair.loc[(markup_tair[0] == chr_name) & (markup_tair[3] == '+')]))

  1%|▋                                                                            | 30/3148 [00:00<00:00, 25642.78it/s]

True
True
True
True
True


In [11]:
markup_tair = markup_tair.loc[(markup_tair[0] == 'chr1') & (markup_tair[3] == '+')]

In [12]:
get_intrscts(markup_tair, markup_real_2, ask_if=True)

  0%|                                                                                | 4/3558 [00:00<00:30, 117.71it/s]


True

In [13]:
df_intrscts = build_intrscts(markup_real_2, markup_tair)

100%|█████████████████████████████████████████████████████████████████████████████| 6196/6196 [00:38<00:00, 161.95it/s]

Общая разметка составляет  0.011409004729573523 % и  0.002160651162638791 % от исходных


In [14]:
df_intrscts.head()

,0,1,2
0,chr1,55331,55668
1,chr1,88271,88615
2,chr1,95728,95919
3,chr1,95728,95919
4,chr1,99326,99619


In [16]:
is_slf_intrsct(df_intrscts)

  0%|                                                                                         | 3/2935 [00:00<?, ?it/s]


True

In [22]:
df_intrscts.to_csv('ciscross_2/ups2000_tair10_intrsct.bed', header=None, index=None, sep='\t')

## Тестирование

In [45]:
markup_tair = pd.read_csv("списки генов/ups2000_tair10.bed", sep="\t", header=None)
markup_tair = markup_tair.loc[(markup_tair[0] == 'chr1')]

In [46]:
markup_real_2 = pd.read_csv("GSM2704255_ATAC_seq/GSM2704255_ATAC_chr1.bed", sep="\t", header=None)

In [52]:
df_intrscts = build_intrscts(markup_real_2, markup_tair)

100%|██████████████████████████████████████████████████████████████████████████████| 6196/6196 [01:11<00:00, 86.15it/s]

Общая разметка составляет  0.011487534786400981 % и  0.0021755232801116253 % от исходных


In [48]:
control_markup = pd.read_csv("Intersections_contr.bed", sep="\t", header=None)

In [54]:
df_intrscts[1]

0          33153
1          37877
2          55331
3          88271
4          88271
          ...   
5843    30389342
5844    30389342
5845    30389342
5846    30409738
5847    30409738
Name: 1, Length: 5848, dtype: object

In [58]:
list(control_markup.sort_values(by=[1], ascending=True)[1]) == list(df_intrscts[1])

True

## Работа с данными, фильтрация

In [29]:
markup_tair = pd.read_csv("списки генов/ups2000_tair10.bed", sep="\t", header=None)

In [30]:
for chr_name in ['chr1', 'chr2', 'chr3', 'chr4', 'chr5']:
    markup_tair.loc[(markup_tair[0] == chr_name)].to_csv('ciscross_2/ups2000_tair10_'+chr_name+'.bed', header=None, index=None, sep='\t')  

In [33]:
markup_tair = pd.read_csv("списки генов/ups2000_tair10_DEG.bed", sep="\t", header=None)

In [34]:
for chr_name in ['chr1', 'chr2', 'chr3', 'chr4', 'chr5']:
    markup_tair.loc[(markup_tair[0] == chr_name)].to_csv('ciscross_2/ups2000_tair10_DEG_'+chr_name+'.bed', header=None, index=None, sep='\t')  

In [35]:
markup_tair = pd.read_csv("списки генов/ups2000_tair10_notDEG.bed", sep="\t", header=None)

In [36]:
for chr_name in ['chr1', 'chr2', 'chr3', 'chr4', 'chr5']:
    markup_tair.loc[(markup_tair[0] == chr_name)].to_csv('ciscross_2/ups2000_tair10_notDEG_'+chr_name+'.bed', header=None, index=None, sep='\t')  

In [39]:
ein3_markup = pd.read_csv("bed files MACS2 processing/EIL_tnt.EIN3_col_a.bed", sep="\t", header=None)

In [40]:
for chr_name in ['chr1', 'chr2', 'chr3', 'chr4', 'chr5']:
    ein3_markup.loc[(ein3_markup[0] == chr_name)].to_csv('ciscross_2/EIL_tnt.EIN3_col_a_'+chr_name+'.bed', header=None, index=None, sep='\t')  

In [59]:
ein3_markup = pd.read_csv("bed files MACS2 processing/EIL_tnt.EIN3_colamp_a.bed", sep="\t", header=None)

In [60]:
for chr_name in ['chr1', 'chr2', 'chr3', 'chr4', 'chr5']:
    ein3_markup.loc[(ein3_markup[0] == chr_name)].to_csv('ciscross_2/EIL_tnt.EIN3_colamp_a_'+chr_name+'.bed', header=None, index=None, sep='\t')  

_________________________________________________________

In [6]:
for chr_name in ['chr1', 'chr2', 'chr3', 'chr4', 'chr5']:
    
    markup_TF = pd.read_csv('ciscross_2/EIL_tnt.EIN3_col_a_'+chr_name+'.bed', sep="\t", header=None)
    markup_ATAC = pd.read_csv('ciscross_2/GSM2704255_ATAC_'+chr_name+'.bed', sep="\t", header=None)
    df_intrscts = build_intrscts(markup_TF, markup_ATAC)
    df_intrscts.to_csv('ciscross_2/EIN3_col_a_vs_ATAC_'+chr_name+'.bed', header=None, index=None, sep='\t')
    
    markup_TF = pd.read_csv('ciscross_2/EIL_tnt.EIN3_colamp_a_'+chr_name+'.bed', sep="\t", header=None)
    df_intrscts = build_intrscts(markup_TF, markup_ATAC)
    df_intrscts.to_csv('ciscross_2/EIN3_colamp_a_vs_ATAC_'+chr_name+'.bed', header=None, index=None, sep='\t')

  5%|████                                                                             | 10/197 [00:00<00:01, 94.27it/s]

Общая разметка составляет  0.0022199768070772408 % и  0.0011170250199758756 % от исходных


 10%|████████                                                                        | 18/180 [00:00<00:00, 174.37it/s]

Общая разметка составляет  0.0025821334234341084 % и  0.002890842538469576 % от исходных


 15%|████████████▏                                                                   | 18/118 [00:00<00:00, 158.28it/s]

...

Общая разметка составляет  0.004720987889331797 % и  0.005401503501812046 % от исходных


  9%|██████▊                                                                         | 16/187 [00:00<00:01, 143.01it/s]

Общая разметка составляет  0.0028909075556855916 % и  0.0014829556964071995 % от исходных


100%|███████████████████████████████████████████████████████████████████████████████| 187/187 [00:01<00:00, 130.35it/s]

Общая разметка составляет  0.003602307773945005 % и  0.004058693561357427 % от исходных
